In [115]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

In [116]:
raw_data_new= pd.read_csv('cleaned_university_data.csv')

In [118]:
raw_data_new['delivery_method'].unique()

array(['Hybrid', 'In-Person', 'Distance'], dtype=object)

In [119]:
conn = psycopg2.connect(
host="localhost",
database="db",
user="user",
password="password",
port=5435
)

In [120]:
cursor = conn.cursor()

In [121]:
raw_data_new.columns

Index(['rank', 'institution', 'year_founded', 'affiliation', 'delivery_method',
       'enrollment', 'city', 'country', 'continent', 'link'],
      dtype='object')

In [122]:
delivery_method_dim= raw_data_new[['delivery_method']].drop_duplicates()
delivery_method_dim['delivery_method_id'] = range(1, len(delivery_method_dim) + 1)
delivery_method_dim = delivery_method_dim [['delivery_method_id', 'delivery_method']]

In [ ]:
institution_dim= raw_data_new[['institution','year_founded','affiliation','link']].drop_duplicates()
institution_dim['institution_id'] = range(1, len(institution_dim) + 1)
institution_dim = institution_dim [['institution_id','institution','year_founded','affiliation','link']]
institution_dim

In [139]:
institution_dim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   institution_id  90 non-null     int64 
 1   institution     90 non-null     object
 2   year_founded    90 non-null     int64 
 3   affiliation     90 non-null     object
 4   link            90 non-null     object
dtypes: int64(2), object(3)
memory usage: 3.6+ KB


In [ ]:
location_dim= raw_data_new[['city','country','continent']].drop_duplicates()
location_dim['location_id'] = range(1, len(location_dim) + 1)    
location_dim = location_dim [['location_id','city','country','continent']]  
location_dim


In [ ]:
university_fact_dm = raw_data_new[['rank', 'enrollment']]
university_fact_dm['university_id'] = range(1, len(university_fact_dm) + 1)
university_fact_dm = university_fact_dm [['university_id','rank', 'enrollment']]
university_fact_dm  

In [101]:
create_query_file=open('create_table.sql', 'r')
create_query = create_query_file.read()
create_query_file

<_io.TextIOWrapper name='create_table.sql' mode='r' encoding='cp1252'>

In [ ]:
cursor.execute(create_query)    
conn.commit()

In [113]:
data_delivery =list(delivery_method_dim.itertuples(index=False, name=None)) 
merge_delivery_query = open('populate_delivery_method.sql', 'r')
merge_query = merge_delivery_query.read()


In [114]:
cursor.executemany(merge_query, data_delivery)
conn.commit()

In [142]:
data_institution =list(institution_dim.itertuples(index=False, name=None)) 
merge_institution_query = open('populate_institution.sql', 'r')
merge_query2 = merge_institution_query.read()

In [140]:
conn.rollback()

In [143]:
cursor.executemany(merge_query2, data_institution)
conn.commit()

InvalidColumnReference: there is no unique or exclusion constraint matching the ON CONFLICT specification
